<a href="https://colab.research.google.com/github/justinpinkney/clip2latent/blob/demo/colab-gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/justinpinkney/clip2latent.git
%cd /content/clip2latent

In [ ]:
!pip install -r requirements-colab.txt
!pip install -e .

In [3]:
from scripts import demo
demo.main()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://21364.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces
